In [1]:
import os
directory = os.getcwd()  
directory = directory.split('src')
path = directory[0]
%cd $path
%pwd

c:\Users\JENMOSQU\Documents\PERSONAL\cod\Probabilistic-Electricity-Demand-


'c:\\Users\\JENMOSQU\\Documents\\PERSONAL\\cod\\Probabilistic-Electricity-Demand-'

In [2]:
#Libraries...........

import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go
from itertools import product
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error, mean_absolute_percentage_error

# # statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX

#Modules from repo
from src.Utilities import utilities as tool

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
# Configuración options.display
# ==============================================================================
pd.set_option('display.max_rows', 5000)
pd.options.display.max_rows = None
pd.options.display.max_columns = 5000
pd.options.display.max_colwidth = 5000
pd.options.display.max_dir_items = 5000
pd.options.display.width = 5000
pd.options.display.float_format = '{:.2f}'.format

Using TensorFlow backend


In [3]:
# Import Demand Data ...........
ruta_directorio = 'C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/Demanda diaria/DEMANDA'
datos_hoja_real = tool.demand_utilities.import_demand(ruta_directorio)

In [4]:
#SUM localidades...........

print("--------Procesando demanda SUM localidades--------")
demanda = [pd.DataFrame(datos) for datos in datos_hoja_real]
mapeo_columnas = {"TIPO DIA": "TIPO_DIA","TOTAL": "Total"}
for df in demanda[:1500]:
    df.rename(columns=mapeo_columnas, inplace=True)
demanda = pd.concat(demanda, axis=0)
valores_a_excluir = ['SINCOLOMBIA', 'MC-SIN']  
columna = 'UCP' 
demanda=demanda[~demanda[columna].isin(valores_a_excluir)]
demanda['FECHA'] = pd.to_datetime(demanda['FECHA'], format='%d/%m/%Y', errors='coerce')
demanda= demanda[['UCP','FECHA','TIPO_DIA','Total']]
demanda=demanda.groupby(['FECHA', 'TIPO_DIA'])['Total'].sum().reset_index()  #Aquí sumo y agrupo por día, todas menos la SIN.

--------Procesando demanda SUM localidades--------


In [5]:
#SIN...........
print("--------Procesando demanda SIN Nueva--------")
demanda_1 = [pd.DataFrame(datos) for datos in datos_hoja_real]
mapeo_columnas = {"TIPO DIA": "TIPO_DIA","TOTAL": "Total"}
for df in demanda_1[:2500]:
    df.rename(columns=mapeo_columnas, inplace=True)
demanda_1 = pd.concat(demanda_1, axis=0)
valores_a_incluir= ['SINCOLOMBIA', 'MC-SIN']  
columna = 'UCP' 
demanda_1=demanda_1[demanda_1[columna].isin(valores_a_incluir)] #los incluye
demanda_1.reset_index(inplace=True)
demanda_1['FECHA'] = demanda_1['FECHA'].astype(str)
demanda_1['FECHA'] = demanda_1['FECHA'].apply(tool.exploratory_tools.reemplazar_guion)
demanda_1['FECHA'] = demanda_1['FECHA'].str.split(' ').str[0]
demanda_1['FECHA'] = pd.to_datetime(demanda_1['FECHA'], dayfirst=True).dt.strftime('%d/%m/%Y')
#demanda_1['FECHA'] = pd.to_datetime(demanda_1['FECHA'], format='%d/%m/%Y')
demanda_1['FECHA'] = pd.to_datetime(demanda_1['FECHA'], format='%d/%m/%Y', errors='coerce')
demanda_1=demanda_1.groupby(['FECHA', 'TIPO_DIA'])['Total'].sum().reset_index() #Aquí sumo y agrupo por día, solamenta la SIN.
print(demanda_1.tail(2))


# print("--------diferencia entre la demanda agregada y la SIN NUEVA--------")
# diferencias = demanda['Total'] - demanda_1['Total']
# print(diferencias[0:5])

--------Procesando demanda SIN Nueva--------
          FECHA TIPO_DIA     Total
2128 2023-09-29  VIERNES 235929.69
2129 2023-09-30   SABADO 228839.63


In [6]:
#Procesando demanda SIN Antigua...........
#Lee los archivos de Excel de demanda de energía en el sistema interconectado nacional (SIN) de una ruta determinada y devuelve una única tabla consolidada

print("--------Procesando demanda SIN Antigua--------")
ruta = "src/Data"
demand = tool.demand_utilities.read_demand_data(ruta)
demand.columns=demand.iloc[2]
demand.drop([0,1,2], axis=0, inplace=True)
demand = demand[demand.apply(tool.demand_utilities.filtro_fechas, axis=1)]
demand['Fecha'] = demand['Fecha'].str.replace('/', '-')
demand['Fecha']=pd.to_datetime(demand['Fecha'])
demand = demand.set_index('Fecha')
demand = demand.asfreq('D')
demand = demand.reset_index()
demand.drop(['Exportaciones kWh','Importaciones kWh','Demanda No Atendida kWh','Generación kWh'], axis=1, inplace=True)
demand.rename(columns={'Demanda Energia SIN kWh':'demanda'}, inplace=True)
demand['demanda'] = demand['demanda'].astype(float)
demand = demand.set_index('Fecha')
demand.reset_index(inplace=True)
demand=demand[(demand['Fecha'] >= '2017-12-01') & (demand['Fecha'] <= '2023-07-31')]
demand.rename(columns={'Fecha':'FECHA'}, inplace= True)
print(demand.shape)

--------Procesando demanda SIN Antigua--------
(2059, 2)


In [7]:
#Comparación de demandas...........
resta = demanda_1['Total'].sub(demand['demanda'], fill_value=0)
print("--------resta--------")
print(resta[0:2])
print("--------Obteniendo las tres demandas unidas--------")
df_merged = demanda.merge(demanda_1, on='FECHA', how='outer').merge(demand, on='FECHA', how='outer')
df_merged.rename(columns={'Total_x':'Demanda_Sum_localidades','Total_y':'Demanda_SIN','demanda':'demanda_SIN_antigua','TIPO_DIA_y':'TIPO_DIA'}, inplace=True)
df_merged.drop(columns={'TIPO_DIA_x'}, inplace=True)
print("--------Obteniendo SIN en la misma Unidad--------")
df_merged['demanda_SIN_antigua'] = pd.to_numeric(df_merged['demanda_SIN_antigua'], errors='coerce')
df_merged['demanda_SIN_antigua'] = df_merged['demanda_SIN_antigua']/ 1000
print(df_merged['demanda_SIN_antigua'][0:2])
print(df_merged.shape)
print(df_merged.head(2))
print(df_merged.tail(2))

--------resta--------
0   193865.75
1   184393.75
dtype: float64
--------Obteniendo las tres demandas unidas--------
--------Obteniendo SIN en la misma Unidad--------
0   193867.76
1   184410.42
Name: demanda_SIN_antigua, dtype: float64
(2130, 5)
       FECHA  Demanda_Sum_localidades TIPO_DIA  Demanda_SIN  demanda_SIN_antigua
0 2017-12-01                379922.18  VIERNES    193865.75            193867.76
1 2017-12-02                359998.30   SABADO    184393.75            184410.42
          FECHA  Demanda_Sum_localidades TIPO_DIA  Demanda_SIN  demanda_SIN_antigua
2128 2023-09-29                      NaN  VIERNES    235929.69                  NaN
2129 2023-09-30                      NaN   SABADO    228839.63                  NaN


In [8]:
# Gráfica de las tres demandas

trace1 = go.Scatter(
    x=df_merged['FECHA'],
    y=df_merged['Demanda_SIN'],
    mode='lines+markers',  # Esto agrega marcadores a la línea
    line=dict(shape='linear', color='#c5899a'),  # Personaliza el estilo de la línea
    name='Demanda_SIN'  # Nombre de la serie para la leyenda
)

# A continuación, puedes seguir agregando otras series si es necesario, utilizando el mismo enfoque con go.Scatter.

# Supongamos que también agregas la segunda serie con go.Scatter
trace2 = go.Scatter(
    x=df_merged['FECHA'],
    y=df_merged['Demanda_Sum_localidades'],
    mode='lines',
    line=dict(color='#5f9ea0'),
    name='Demanda_SUMA_localidades'
)

# Agrega la tercera serie con go.Scatter
trace3 = go.Scatter(
    x=df_merged['FECHA'],
    y=df_merged['demanda_SIN_antigua'],
    mode='lines',
    line=dict(color='#793f12'),
    name='Demanda_SIN_antigua'
)

# Ahora puedes crear la figura con todas las trazas
fig = go.Figure(data=[trace1, trace2, trace3])

# Personaliza el diseño de la leyenda
fig.update_layout(
    legend=dict(
        title='Series',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

# Actualiza los títulos de los ejes
fig.update_xaxes(title='Fecha')
fig.update_yaxes(title='demandas_obtenidas')

# Muestra el gráfico
fig.show()



In [9]:
#Gráfica de Demanda SIN Nueva y Antigua

trace1 = go.Scatter(
    x=df_merged['FECHA'],
    y=df_merged['Demanda_SIN'],
    mode='lines+markers',  # Esto agrega marcadores a la línea
    line=dict(shape='linear', color='#c5899a'),  # Personaliza el estilo de la línea
    name='Demanda_SIN'  # Nombre de la serie para la leyenda
)

# Agrega la tercera serie con go.Scatter
trace3 = go.Scatter(
    x=df_merged['FECHA'],
    y=df_merged['demanda_SIN_antigua'],
    mode='lines',
    line=dict(color='#793f12'),
    name='Demanda_SIN_antigua'
)

# Ahora puedes crear la figura con todas las trazas
fig = go.Figure(data=[trace1, trace3])

# Personaliza el diseño de la leyenda
fig.update_layout(
    legend=dict(
        title='Series',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

# Actualiza los títulos de los ejes
fig.update_xaxes(title='Fecha')
fig.update_yaxes(title='demandas_obtenidas')

# Muestra el gráfico
fig.show()

In [10]:
# Definir completitud de días...........

fecha_inicio = datetime(2017, 12, 1)
fecha_fin = datetime(2023, 9, 30)
# Calcular la diferencia entre las fechas
diferencia = relativedelta(fecha_fin, fecha_inicio)
# Calcular el número total de días en la diferencia
dias_totales = diferencia.years * 365 + diferencia.months * 30 + diferencia.days
print(f"El número total de días entre las fechas es: {dias_totales} días")


# Verificar si hay días repetidos
dias_repetidos = demand['FECHA'].duplicated()
# Mostrar el resultado
if dias_repetidos.any():
    print("Se encontraron días repetidos:")
    print(df[dias_repetidos])
else:
    print("No se encontraron días repetidos.")

El número total de días entre las fechas es: 2124 días
No se encontraron días repetidos.


In [11]:
# SIN...........
#---------------------------------------------------------------
print("--------Obteniendo Lista de fechas Outliers------------------------------------------------------------------------")
outliers_list=demanda_1.nlargest(3, 'Total')['FECHA'].to_list()
outliers_list=[i.to_numpy() for i in outliers_list]
print(outliers_list)
#---------------------------------------------------------------

print("--------Imputando la data nueva con la lista de fechas Outliers desde la SIN antigua------------------------------------------------------------------------")
df_merged.loc[df_merged['FECHA'].isin(outliers_list), 'Demanda_SIN'] = df_merged['demanda_SIN_antigua']
df_merged[df_merged['FECHA'].isin(outliers_list)]
demanda= df_merged[['FECHA','TIPO_DIA','Demanda_SIN']]

print("--------Obteniendo Variables de control: Covid, cambio de nivel, tiempo------------------------------------------------------------------------")
demanda['Covid'] = (demanda['FECHA'] >= '2020-03-19') & (demanda['FECHA'] <= '2021-07-23')
demanda['Cambio_nivel'] = (demanda['FECHA'] >= '2020-03-19')
demanda['Tiempo'] = range(1, len(demanda) + 1)
demanda['Covid'] = demanda['Covid'].astype(int)
demanda['Cambio_nivel'] = demanda['Cambio_nivel'].astype(int)
print(demanda.head(2))
print(demanda.tail(2))
# demanda.to_parquet('demanda.parquet')


--------Obteniendo Lista de fechas Outliers------------------------------------------------------------------------
[numpy.datetime64('2020-01-27T00:00:00.000000000'), numpy.datetime64('2020-01-25T00:00:00.000000000'), numpy.datetime64('2020-01-26T00:00:00.000000000')]
--------Imputando la data nueva con la lista de fechas Outliers desde la SIN antigua------------------------------------------------------------------------
--------Obteniendo Variables de control: Covid, cambio de nivel, tiempo------------------------------------------------------------------------
       FECHA TIPO_DIA  Demanda_SIN  Covid  Cambio_nivel  Tiempo
0 2017-12-01  VIERNES    193865.75      0             0       1
1 2017-12-02   SABADO    184393.75      0             0       2
          FECHA TIPO_DIA  Demanda_SIN  Covid  Cambio_nivel  Tiempo
2128 2023-09-29  VIERNES    235929.69      0             1    2129
2129 2023-09-30   SABADO    228839.63      0             1    2130


In [12]:
#Gráfica de Demanda SIN Nueva y Antigua

trace1 = go.Scatter(
    x=df_merged['FECHA'],
    y=df_merged['Demanda_SIN'],
    mode='lines+markers',  # Esto agrega marcadores a la línea
    line=dict(shape='linear', color='#c5899a'),  # Personaliza el estilo de la línea
    name='Demanda_SIN'  # Nombre de la serie para la leyenda
)

# Agrega la tercera serie con go.Scatter
trace2 = go.Scatter(
    x=df_merged['FECHA'],
    y=df_merged['demanda_SIN_antigua'],
    mode='lines',
    line=dict(color='#793f12'),
    name='Demanda_SIN_antigua'
)

# Ahora puedes crear la figura con todas las trazas
fig = go.Figure(data=[trace1, trace2])

# Personaliza el diseño de la leyenda
fig.update_layout(
    legend=dict(
        title='Series',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

# Actualiza los títulos de los ejes
fig.update_xaxes(title='Fecha')
fig.update_yaxes(title='demandas_obtenidas')

# Muestra el gráfico
fig.show()

In [13]:
#Gráfica de Demanda SIN Nueva...................................

trace1 = go.Scatter(
    x=df_merged['FECHA'],
    y=df_merged['Demanda_SIN'],
    mode='lines+markers',  # Esto agrega marcadores a la línea
    line=dict(shape='linear', color='#276981'), 
    name='Demanda_SIN' 
)
fig = go.Figure(data=[trace1])
fig.update_layout(
    legend=dict(
        title='Series',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)
fig.update_xaxes(title='Date-daily')
fig.update_yaxes(title='Electricity Demand SIN kWh')
fig.show()

In [12]:
#Matriz de rezagos: tool.demand_utilities.generar_base_rezagos()
n_rezagos = 28
df_rezagos = tool.demand_utilities.generar_base_rezagos(demanda.copy(), n_rezagos)
df_rezagos.head(2)

,FECHA,TIPO_DIA,Demanda_SIN,Covid,Cambio_nivel,Tiempo,Demanda_SIN_1,Demanda_SIN_2,Demanda_SIN_3,Demanda_SIN_4,Demanda_SIN_5,Demanda_SIN_6,Demanda_SIN_7,Demanda_SIN_8,Demanda_SIN_9,Demanda_SIN_10,Demanda_SIN_11,Demanda_SIN_12,Demanda_SIN_13,Demanda_SIN_14,Demanda_SIN_15,Demanda_SIN_16,Demanda_SIN_17,Demanda_SIN_18,Demanda_SIN_19,Demanda_SIN_20,Demanda_SIN_21,Demanda_SIN_22,Demanda_SIN_23,Demanda_SIN_24,Demanda_SIN_25,Demanda_SIN_26,Demanda_SIN_27,Demanda_SIN_28
0,2017-12-01,VIERNES,193865.75,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-12-02,SABADO,184393.75,0,0,2,193865.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Datos de clima

* PTPG_TT_D: Precipitación total diaria
* TSSM_MX_D: Temperatura seca máxima diaria
* TSSM_MN_D: Temperatura seca mínima diaria
* TSSM_MEDIA_D: Temperatura seca media diaria
* PT_10_TT_D: Precipitación total diaria
* PT_10_MX_D: Precipitación máxima diaria
* PT_2_MX_D: Precipitación máxima diaria
* THSM_MN_D: Temperatura húmeda mínima diaria
* VVAG_MEDIA_D: Velocidad del viento media diaria
* VV_10_VECT_MEDIA_D: Persistencia diaria
* VV_10_MEDIA_D: Velocidad del viento media diaria
* THSM_MX_D: Temperatura húmeda máxima diaria
* THSM_MEDIA_D: Temperatura húmeda máxima diaria
* HRA2_MX_D: Humedad relativa del aire a 2 metros máxima diaria
* HRA2_MN_D: Humedad relativa del aire a 2 metros mínima diaria
* HR_CAL_MEDIA_D: Humedad Relativa media diaria
* BSHG_TT_D: Brillo solar total diario
* TPR_CAL: Temperatura del punto de roció media diaria
* HRA2_AUT:Humedad relativa del aire a 2 metros media diaria AUTOMATICA
* HRA10_MEDIA_D: Humedad relativa del aire a 10 cm media diaria.
* EV_TT_D: Evaporación total diaria
* TA2_MEDIA_D: Temperatura del Aire a 2 metros media diaria

In [13]:
# Datos del clima nuevos de prep y temp..................

#Antiguas
#...........................................................................................................................................
# filepath_prep = "C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/precipitación/precip.parquet"
# filepath_temp = "C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/temperatura/temperatura.parquet"
#...........................................................................................................................................


#Nuevas.....................................................................................................................................
filepath_prep='C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/PrecipitacionNacionalDiaria/precip.parquet'
filepath_temp="C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/TemperaturaNacionalDiaria/temperatura.parquet"
#...........................................................................................................................................
column = "Fecha"
resampling = "D" #diario/ "D" para días, "H" para horas

df_resampled_prep = tool.demand_utilities.read_parquet_file_and_resample(filepath_prep, column, resampling)
df_resampled_temp = tool.demand_utilities.read_parquet_file_and_resample(filepath_temp, column, resampling)

df_resampled_prep.reset_index(inplace=True)
df_resampled_temp.reset_index(inplace=True)

# Filtrar el DataFrame desde una fecha específica
fecha_inicio = pd.to_datetime("2017-12-01")  # Convierte la fecha en formato datetime si aún no lo está
df_resampled_prep = df_resampled_prep[df_resampled_prep["Fecha"] >= fecha_inicio]
df_resampled_temp=df_resampled_temp[df_resampled_temp["Fecha"] >= fecha_inicio]
print(df_resampled_prep.tail(2))
print(df_resampled_temp.tail(2))

          Fecha  Valor
8556 2023-06-05   8.46
8557 2023-06-06   1.57
          Fecha  Valor
8556 2023-06-05  21.99
8557 2023-06-06  19.52


In [13]:
#Gráfica de Precipitación y temperatura Nueva...........

trace1 = go.Scatter(
    x=df_resampled_prep['Fecha'],
    y=df_resampled_prep['Valor'],
    mode='lines+markers',  # Esto agrega marcadores a la línea
    line=dict(shape='linear', color='#256d7b'),  # Personaliza el estilo de la línea
    name='Precipitación Nueva'  # Nombre de la serie para la leyenda
)

# Agrega la tercera serie con go.Scatter
trace2 = go.Scatter(
    x=df_resampled_temp['Fecha'],
    y=df_resampled_temp['Valor'],
    mode='lines',
    line=dict(color='#009c8c'),
    name='Temperatura Nueva'
)

# Ahora puedes crear la figura con todas las trazas
fig = go.Figure(data=[trace1, trace2])

# Personaliza el diseño de la leyenda
fig.update_layout(
    legend=dict(
        title='Series',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

# Actualiza los títulos de los ejes
fig.update_xaxes(title='Fecha')
fig.update_yaxes(title='Series del clima antiguas')

# Muestra el gráfico
fig.show()

In [15]:
#Precipitación y temperatura........
#-----------------------------------------------------------------------------------------
print("Uniendo Precipitación y temperatura")
df_resampled_prep.rename(columns={'Fecha':'FECHA'}, inplace=True)
df_resampled_temp.rename(columns={'Fecha':'FECHA'}, inplace=True)
df = pd.merge(df_rezagos, df_resampled_prep, on='FECHA', how='outer')
df.rename(columns={'Valor':'precipitacion'}, inplace=True)
df = pd.merge(df, df_resampled_temp, on='FECHA', how='outer')
df.rename(columns={'Valor':'Temperatura'}, inplace=True)
print(df.head(2))
print(df.tail(2))

window_size = 60

Uniendo Precipitación y temperatura
       FECHA TIPO_DIA  Demanda_SIN  Covid  Cambio_nivel  Tiempo  Demanda_SIN_1  Demanda_SIN_2  Demanda_SIN_3  Demanda_SIN_4  Demanda_SIN_5  Demanda_SIN_6  Demanda_SIN_7  Demanda_SIN_8  Demanda_SIN_9  Demanda_SIN_10  Demanda_SIN_11  Demanda_SIN_12  Demanda_SIN_13  Demanda_SIN_14  Demanda_SIN_15  Demanda_SIN_16  Demanda_SIN_17  Demanda_SIN_18  Demanda_SIN_19  Demanda_SIN_20  Demanda_SIN_21  Demanda_SIN_22  Demanda_SIN_23  Demanda_SIN_24  Demanda_SIN_25  Demanda_SIN_26  Demanda_SIN_27  Demanda_SIN_28  precipitacion  Temperatura
0 2017-12-01  VIERNES    193865.75      0             0       1            NaN            NaN            NaN            NaN            NaN            NaN            NaN            NaN            NaN             NaN             NaN             NaN             NaN             NaN             NaN             NaN             NaN             NaN             NaN             NaN             NaN             NaN             NaN           

In [16]:
for i in range(len(df['precipitacion'])):
    if pd.isnull(df['precipitacion'].iloc[i]):
        if i >= window_size:
            df['precipitacion'].iloc[i] = df['precipitacion'].iloc[i - window_size:i].mean()
        else:
            df['precipitacion'].iloc[i] = df['precipitacion'].iloc[:i].mean()




for i in range(len(df['Temperatura'])):
    if pd.isnull(df['Temperatura'].iloc[i]):
        if i >= window_size:
            df['Temperatura'].iloc[i] = df['Temperatura'].iloc[i - window_size:i].mean()
        else:
            df['Temperatura'].iloc[i] = df['Temperatura'].iloc[:i].mean()

In [17]:
#Datos de clima... ANTERIORES PERO MÁS.................

ruta_archivos = "C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894"

elementos=['HRA2_MN_D','HRA2_MX_D','PT_10_MX_D','PT_10_TT_D','PT_2_MX_D','PT_2_TT_D','PTPG_TT_D','TA10_MX_10_D','TAMX2_MN_D','TA10_MEDIA_D','TA10_MN_D',
           'TA10_MX_D','TAMN10_MEDIA_D','TAMN10_MN_D','TAMN10_MX_D','TAMX10_MEDIA_D','TAMX10_MN_D','TAMX10_MX_10_D','TAMX10_MX_D','TAMX2_MEDIA_D','TAMX2_MX_10_D',
           'TAMX2_MX_D','THSM_MEDIA_D','THSM_MN_D','THSM_MX_D','TSSM_MEDIA_D','TSSM_MN_D','TSSM_MX_D','TSTG_MEDIA_D','TSTG_MN_D','TSTG_MX_D','VV_10_MEDIA_D',
           'VV_10_VECT_MEDIA_D','VVAG_MEDIA_D','VVMX_MX_D']
dfs = {}  
filepaths = {}

for elemento in elementos:
    tool.demand_utilities.filtrar_exportar_parquet(ruta_archivos, "2017-12-01", "2023-09-30", elemento, f"{elemento}@")
    print(f"Archivo parquet de {elemento} es guardado en:", os.path.join(ruta_archivos, f"{elemento}.parquet"))
    filepath_elemento = os.path.join(ruta_archivos, f"{elemento}.parquet")
    column = "Fecha"
    resampling = "D" 
    df_resampled_elemento = tool.demand_utilities.read_parquet_file_and_resample(filepath_elemento, column, resampling)
    df_resampled_elemento.reset_index(inplace=True)
    fecha_inicio = pd.Timestamp("2017-12-01")
    fecha_fin = pd.Timestamp("2023-08-30")
    dfs[elemento] = df_resampled_elemento
    filepaths[elemento] = filepath_elemento

Archivo parquet de HRA2_MN_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\HRA2_MN_D.parquet
Archivo parquet de HRA2_MX_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\HRA2_MX_D.parquet
Archivo parquet de PT_10_MX_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\PT_10_MX_D.parquet
Archivo parquet de PT_10_TT_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\PT_10_TT_D.parquet
Archivo parquet de PT_2_MX_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\PT_2_MX_D.parquet
Archivo parquet de PT_2_TT_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894\PT_2_TT_D.parquet
Archivo parquet de PTPG_TT_D es guardado en: C:/Users/JENMOS

In [ ]:
['VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D']

In [ ]:
'C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894/VV_10_MEDIA_D.parquet',
'C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894/VV_10_VECT_MEDIA_D.parquet',
'C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894/PT_10_TT_D.parquet',
'C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894/THSM_MEDIA_D.parquet'



In [2]:
import pandas as pd

In [9]:
VV_10_MEDIA_D=pd.read_parquet('C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894/VV_10_MEDIA_D.parquet')
VV_10_MEDIA_D.tail()

VV_10_MEDIA_D[VV_10_MEDIA_D['Fecha']>'2023-07-30 07:00:00']

,Fecha,Valor
2170,2023-07-31,1.366667
2627,2023-07-31,2.967708
1833,2023-07-31,7.943889
2806,2023-07-31,3.535079
2466,2023-07-31,1.911285
2105,2023-07-31,1.239902
1647,2023-07-31,2.159783
2023,2023-07-31,2.406597
2505,2023-07-31,673.899306
3041,2023-07-31,2.168958


In [8]:
VV_10_VECT_MEDIA_D=pd.read_parquet('C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/RAD_20239050055724_20239910055894/VV_10_VECT_MEDIA_D.parquet')
VV_10_VECT_MEDIA_D.tail()
VV_10_VECT_MEDIA_D[VV_10_VECT_MEDIA_D['Fecha']>'2023-07-30 07:00:00']

,Fecha,Valor
2154,2023-07-31,0.597336
2615,2023-07-31,1.456262
1826,2023-07-31,7.686025
2787,2023-07-31,2.966349
2463,2023-07-31,0.111325
2100,2023-07-31,0.393602
1633,2023-07-31,1.887017
2013,2023-07-31,0.273204
2420,2023-07-31,673.603870
2377,2023-07-31,1.216746


In [18]:
print("--------Formateando columnas del clima --------")
df_weather = pd.concat(dfs.values(), axis=1, keys=dfs.keys())
# Asegurarse de que las fechas sean únicas en el DataFrame resultante
df_weather = df_weather.loc[:, ~df_weather.columns.duplicated()]
df_weather.columns = df_weather.columns.map('_'.join)
df_weather.rename(columns={'Fecha_Fecha': 'Fecha', 'TSSM_MEDIA_D_Fecha':'FECHA'},inplace=True)
df_weather.drop(columns=['HRA2_MN_D_Fecha','TAMX2_MX_D_Fecha','VVMX_MX_D_Fecha','TSTG_MX_D_Fecha','TSTG_MN_D_Fecha','TSTG_MEDIA_D_Fecha',
                         'TSSM_MX_D_Fecha','TSSM_MN_D_Fecha','THSM_MX_D_Fecha','THSM_MX_D_Fecha','THSM_MN_D_Fecha','THSM_MEDIA_D_Fecha',
                         'TAMX2_MX_10_D_Fecha','TAMX2_MX_10_D_Fecha','TAMX2_MEDIA_D_Fecha', 'TAMX10_MX_D_Fecha','TAMX10_MX_10_D_Fecha',
                         'TAMX10_MN_D_Fecha','TAMX10_MEDIA_D_Fecha','TAMN10_MX_D_Fecha', 'TAMN10_MN_D_Fecha', 'TAMN10_MEDIA_D_Fecha', 
                         'TA10_MX_D_Fecha', 'TA10_MN_D_Fecha', 'TAMN10_MEDIA_D_Fecha', 'TA10_MX_D_Fecha', 'TA10_MN_D_Fecha', 'TA10_MEDIA_D_Fecha', 
                         'TAMX2_MN_D_Fecha', 'TA10_MX_10_D_Fecha', 'HRA2_MX_D_Fecha','PT_10_MX_D_Fecha', 'PT_10_TT_D_Fecha','VV_10_MEDIA_D_Fecha', 
                         'PT_2_MX_D_Fecha', 'PTPG_TT_D_Fecha', 'PT_2_TT_D_Fecha', 'VV_10_VECT_MEDIA_D_Fecha', 'VVAG_MEDIA_D_Fecha'], inplace=True)
print("--------null_percent_list--------")
# Generador 
null_percent=tool.exploratory_tools.null_proportion(data=df_weather)
null_percent_list=null_percent[null_percent['Percent']< 6.0]['Name'].to_list()
print("Null list",null_percent_list)
df_weather = df_weather[null_percent_list]
print("--------Remover FECHA e imputar con ffill--------")
columna_moved = df_weather.pop('FECHA')
df_weather.insert(0, 'FECHA', columna_moved)
df_weather[df_weather['FECHA'].isna()]
indice_observacion_a_actualizar = 2068  
df_weather.at[indice_observacion_a_actualizar, 'FECHA'] = pd.to_datetime('2023-07-31')
df_weather = df_weather[df_weather['FECHA'] <= '2023-08-20']
print(df_weather.shape)
print(df_weather.head(2))
print(df_weather.tail(2))

--------Formateando columnas del clima --------
--------null_percent_list--------
Null list ['HRA2_MN_D_Valor', 'HRA2_MX_D_Valor', 'THSM_MEDIA_D_Valor', 'THSM_MX_D_Valor', 'VV_10_MEDIA_D_Valor', 'VV_10_VECT_MEDIA_D_Valor', 'VVAG_MEDIA_D_Valor', 'THSM_MN_D_Valor', 'PT_2_TT_D_Valor', 'PT_2_MX_D_Valor', 'PT_10_MX_D_Valor', 'PT_10_TT_D_Valor', 'FECHA', 'TSSM_MEDIA_D_Valor', 'TSSM_MN_D_Valor', 'TSSM_MX_D_Valor', 'PTPG_TT_D_Valor']
--------Remover FECHA e imputar con ffill--------
(2089, 17)
       FECHA  HRA2_MN_D_Valor  HRA2_MX_D_Valor  THSM_MEDIA_D_Valor  THSM_MX_D_Valor  VV_10_MEDIA_D_Valor  VV_10_VECT_MEDIA_D_Valor  VVAG_MEDIA_D_Valor  THSM_MN_D_Valor  PT_2_TT_D_Valor  PT_2_MX_D_Valor  PT_10_MX_D_Valor  PT_10_TT_D_Valor  TSSM_MEDIA_D_Valor  TSSM_MN_D_Valor  TSSM_MX_D_Valor  PTPG_TT_D_Valor
0 2017-12-01            58.69            91.53               19.11            20.70                 1.70                      0.93                1.75            17.86             1.77             1.2

In [19]:
#Datos de clima NUEVOS...........

#'TPR_CAL','TV_CAL'
ruta_archivos = "C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/MetereologiaNacionalDiaria"
elementos=['BSHG_TT_D','HR_CAL_MEDIA_D','TPR_CAL']

dfs = {}  
filepaths = {}

for elemento in elementos:
    tool.demand_utilities.filtrar_exportar_parquet(ruta_archivos, "2017-12-01", "2023-09-30", elemento, f"{elemento}@")
    print(f"Archivo parquet de {elemento} es guardado en:", os.path.join(ruta_archivos, f"{elemento}.parquet"))

    filepath_elemento = os.path.join(ruta_archivos, f"{elemento}.parquet")
    column = "Fecha"
    resampling = "D"  # diario/ "D" para días, "H" para horas, "30T" para intervalos de 30 minutos

    df_resampled_elemento = tool.demand_utilities.read_parquet_file_and_resample(filepath_elemento, column, resampling)
    df_resampled_elemento.reset_index(inplace=True)

    # Convierte las cadenas en objetos Timestamp
    fecha_inicio = pd.Timestamp("2017-12-01")
    fecha_fin = pd.Timestamp("2023-09-30")

    # Verifica si el DataFrame no tiene valores NaN en el rango de fechas deseado
    #if df_resampled_elemento["Fecha"].min() <= fecha_inicio and df_resampled_elemento["Fecha"].max() >= fecha_fin and not df_resampled_elemento.isna().values.any():
    dfs[elemento] = df_resampled_elemento
    filepaths[elemento] = filepath_elemento

Archivo parquet de BSHG_TT_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/MetereologiaNacionalDiaria\BSHG_TT_D.parquet
Archivo parquet de HR_CAL_MEDIA_D es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/MetereologiaNacionalDiaria\HR_CAL_MEDIA_D.parquet
Archivo parquet de TPR_CAL es guardado en: C:/Users/JENMOSQU/OneDrive - Grupo Bancolombia/Datos-tesis/CLIMA/nuevo_20231028/MetereologiaNacionalDiaria\TPR_CAL.parquet


In [20]:
print("--------Formateando columnas del clima --------")
df_weather_new = pd.concat(dfs.values(), axis=1, keys=dfs.keys())
df_weather_new = df_weather_new.loc[:, ~df_weather_new.columns.duplicated()]
df_weather_new.columns = df_weather_new.columns.map('_'.join)
df_weather_new.rename(columns={'BSHG_TT_D_Fecha':'FECHA'},inplace=True)
df_weather_new.drop(columns=['HR_CAL_MEDIA_D_Fecha','TPR_CAL_Fecha'], inplace=True)
print("--------null_percent_list--------")
# Generador 
null_percent=tool.exploratory_tools.null_proportion(data=df_weather_new)
null_percent_list=null_percent[null_percent['Percent']< 6.0]['Name'].to_list()
print("Null list",null_percent_list)
df_weather_new = df_weather_new[null_percent_list]
print("--------Remover FECHA e imputar con ffill--------")
columna_moved = df_weather_new.pop('FECHA')
df_weather_new.insert(0, 'FECHA', columna_moved)
df_weather_new[df_weather_new['FECHA'].isna()]
# indice_observacion_a_actualizar = 2068  
# df_weather.at[indice_observacion_a_actualizar, 'FECHA'] = pd.to_datetime('2023-07-31')
df_weather_new = df_weather_new[df_weather_new['FECHA'] <= '2023-06-05']
print(df_weather_new.head(2))
print(df_weather_new.tail(2))

--------Formateando columnas del clima --------
--------null_percent_list--------
Null list ['HR_CAL_MEDIA_D_Valor', 'FECHA', 'BSHG_TT_D_Valor', 'TPR_CAL_Valor']
--------Remover FECHA e imputar con ffill--------
       FECHA  HR_CAL_MEDIA_D_Valor  BSHG_TT_D_Valor  TPR_CAL_Valor
0 2017-12-01                 82.76             4.49          17.98
1 2017-12-02                 82.69             4.93          18.00
          FECHA  HR_CAL_MEDIA_D_Valor  BSHG_TT_D_Valor  TPR_CAL_Valor
1998 2023-05-22                 85.54             5.32          19.92
1999 2023-05-23                 83.76             5.46          20.47
2000 2023-05-24                 81.95             6.34          20.57
2001 2023-05-25                 84.58             3.79          20.74
2002 2023-05-26                 88.21             3.44          20.81
2003 2023-05-27                 83.79             5.85          20.57
2004 2023-05-28                 81.88             5.57          19.41
2005 2023-05-29            

In [21]:
# Unir los DataFrames.............
df_weather_completed = pd.merge(df_weather, df_weather_new, on='FECHA', how='outer')
column_mapping = {col: col.replace('_Valor', '') for col in df_weather_completed.columns if col.endswith('_Valor')}
df_weather_completed.rename(columns=column_mapping, inplace=True)
df = pd.merge(df, df_weather_completed, on='FECHA', how='outer')
df.tail(2)

,FECHA,TIPO_DIA,Demanda_SIN,Covid,Cambio_nivel,Tiempo,Demanda_SIN_1,Demanda_SIN_2,Demanda_SIN_3,Demanda_SIN_4,Demanda_SIN_5,Demanda_SIN_6,Demanda_SIN_7,Demanda_SIN_8,Demanda_SIN_9,Demanda_SIN_10,Demanda_SIN_11,Demanda_SIN_12,Demanda_SIN_13,Demanda_SIN_14,Demanda_SIN_15,Demanda_SIN_16,Demanda_SIN_17,Demanda_SIN_18,Demanda_SIN_19,Demanda_SIN_20,Demanda_SIN_21,Demanda_SIN_22,Demanda_SIN_23,Demanda_SIN_24,Demanda_SIN_25,Demanda_SIN_26,Demanda_SIN_27,Demanda_SIN_28,precipitacion,Temperatura,HRA2_MN_D,HRA2_MX_D,THSM_MEDIA_D,THSM_MX_D,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,VVAG_MEDIA_D,THSM_MN_D,PT_2_TT_D,PT_2_MX_D,PT_10_MX_D,PT_10_TT_D,TSSM_MEDIA_D,TSSM_MN_D,TSSM_MX_D,PTPG_TT_D,HR_CAL_MEDIA_D,BSHG_TT_D,TPR_CAL
2128,2023-09-29,VIERNES,235929.69,0,1,2129,235940.36,238964.91,236506.30,233453.08,203949.28,224268.00,240180.55,239199.92,235739.22,233077.89,226642.06,202119.04,221894.13,233603.48,237458.68,236626.71,237373.32,230019.69,198824.22,225036.01,239583.47,239349.43,239483.36,235227.84,225493.86,199834.17,225256.61,240175.69,7.48,22.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2129,2023-09-30,SABADO,228839.63,0,1,2130,235929.69,235940.36,238964.91,236506.30,233453.08,203949.28,224268.00,240180.55,239199.92,235739.22,233077.89,226642.06,202119.04,221894.13,233603.48,237458.68,236626.71,237373.32,230019.69,198824.22,225036.01,239583.47,239349.43,239483.36,235227.84,225493.86,199834.17,225256.61,7.49,22.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Export Final Data

In [22]:
#df.to_parquet('df.parquet')

#### Demo tool.modeling.optimize_SARIMA

In [12]:
#Demanda sumando todos las localidades pero sin las variables del clima 
y = df['Total']
x=df[['Outliers','Covid','Cambio_nivel','Tiempo']]
d=1
D=1
s=7
parametros = [3,3,3,3]
validacion = 28
forecasting_steps=28
result_1=tool.modeling.optimize_SARIMA(y, x,  d, D, s, parametros, validacion, forecasting_steps)
result_1

  0%|          | 0/256 [00:00<?, ?it/s]

{'forecast': 2041   222421.46
 2042   223644.95
 2043   220215.11
 2044   224264.16
 2045   213684.70
 2046   194635.86
 2047   220343.04
 2048   223782.82
 2049   224304.49
 2050   220270.88
 2051   226033.20
 2052   215358.10
 2053   195829.82
 2054   221152.58
 2055   224565.01
 2056   225371.61
 2057   219480.70
 2058   226446.84
 2059   215979.55
 2060   196834.75
 2061   222148.88
 2062   225075.32
 2063   225630.32
 2064   221856.35
 2065   226768.93
 2066   216821.82
 2067   197529.09
 2068   222012.77
 Name: predicted_mean, dtype: float64,
 'parameters': (3, 1, 3, 0),
 'mape': 3.0038676669079534}

In [ ]:
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=2)
# imputed_data=imputer.fit_transform(df_weather_completed)
# df_weather_completed=pd.DataFrame(imputed_data, columns=df_weather_completed.columns)
# df_weather_completed.tail(10)